In [34]:
import ast

import nltk
#nltk.download('punkt')
from nltk.corpus.reader.plaintext import PlaintextCorpusReader
import re
import pandas as pd
from nltk.corpus import stopwords
#nltk.download('stopwords')
from tqdm import tqdm
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
#import community

In [2]:
df_abstract = pd.read_csv('my_data/df_CSS_paper_abstract.csv')

In [2]:
text = PlaintextCorpusReader('my_data/', '.*')

In [3]:
text.words('test_text.txt')

OSError: No such file or directory: 'C:\\Users\\August\\Documents\\Github\\comsocsci2023\\my_data\\test_text.txt'

In [ ]:
with open('my_data/test_text.txt') as text_file:
    raw = text_file.read()

In [ ]:
tokens = nltk.word_tokenize(raw)
#tokens

In [ ]:
#pattern = r"[^\w\s]"

tokens = [token.lower() for token in tokens]

In [6]:
def remove_digits(l):
    pattern = r'[0-9\p{P}]'
    l = [re.sub(pattern, '', token) for token in l]
    return l

In [ ]:
    tokens = remove_digits(tokens)
tokens

In [7]:
def clean_strings(strings):
    cleaned_strings = []
    stop_words = set(stopwords.words('english'))

    for string in strings:
        # Remove URLs
        string = re.sub(r'http\S+', '', string)

        # Remove numbers
        string = re.sub(r'[0-9]', '', string)

        # Keep only what is not punctuation
        string = re.sub(r'[^\w\s]', '', string)

        # Remove stop words and moke them
        if string not in stop_words: string = string.lower()

        # Remove empty strings
        if len(string):
            cleaned_strings.append(string)

    return cleaned_strings

In [ ]:
print(clean_strings(tokens))

In [9]:
cleaned_corpus = []
df_abstract['tokens'] = None
for i, row in tqdm(df_abstract.iterrows()):
    abstract = row['abstract']
    if type(abstract) == str:

        tokens = nltk.word_tokenize(row['abstract'])
        clean_tokens = clean_strings(tokens)
        cleaned_corpus.append(clean_tokens)
        df_abstract['tokens'][i] = clean_tokens
    else:
        cleaned_corpus.append(None)

0it [00:00, ?it/s]C:\Users\August\AppData\Local\Temp\ipykernel_12496\4017611902.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_abstract['tokens'][i] = clean_tokens
35593it [00:33, 1053.20it/s]


In [3]:
graph = nx.read_graphml('my_data/CSS_graph.graphml')

In [89]:
def louvain_communities(graph):
    # Use the louvain method to find communities
    partition = community.best_partition(graph) # community function that uses Louvain-algorithm

    # Reformat the partitioning
    communities = {}
    for node, community_id in partition.items():
        if community_id in communities:
            communities[community_id].append(node)
        else:
            communities[community_id] = [node]

    return list(communities.values())

In [3]:
communities = np.load('my_data/CSS_louvain_groups.npy', allow_pickle=True)

In [109]:
df_abstract['tokens']


0        [the, term, linked, data, refers, to, a, set, ...
1        [the, term, linked, data, refers, to, a, set, ...
2        [the, term, linked, data, refers, to, a, set, ...
3                                                     None
4        [the, era, of, big, data, has, begun, computer...
                               ...                        
35588    [the, digitization, of, matrimonial, matchmaki...
35589    [the, misuse, of, social, media, platforms, to...
35590    [with, twitter, growing, as, a, preferred, cha...
35591    [with, twitter, growing, as, a, preferred, cha...
35592    [this, chapter, concludes, that, health, promo...
Name: tokens, Length: 35593, dtype: object

In [ ]:
all_communities = []
#Test if pandas explode is more efficient
# Den her kan gøre meget bedre tror jeg, skal bare lige tænke mig lidt om
    for i, row in tqdm(df_abstract.iterrows(), total = df_abstract.shape[0]):
        for community in communities:
            community_tokens = []
            for authorID in community:
                authors = []
                if type(row['authors']) == str:
                    authors = ast.literal_eval(row['authors'])
                if authorID in authors:
                    tokens = None
                    if type(row['tokens']) == str:
                        tokens = ast.literal_eval(row['tokens'])
                    community_tokens.append(tokens)
            all_communities.append(community_tokens)





 20%|██        | 7162/35593 [07:26<29:32, 16.04it/s]  


KeyboardInterrupt: 

In [12]:
community_docs = np.load("my_data/community_documents.npy")

numpy.ndarray

In [4]:
CSS_giant = nx.read_graphml('my_data/CSS_giant.graphml')

In [99]:
# Load the tokens of the community documents if they are already saved
DATA_PATH = "my_data/"

# Create a document for each community that contains the combined abstracts of all papers in that community
community_documents = [[] for _ in range(len(communities))]
"""
Nyt
"""
titles_in_comm = [[] for _ in range(len(communities))] # bruger jeg senere, kan nok gøres smarter
# Create a dictionary that maps nodes to communities
node_to_community = {}
for i, community in enumerate(communities):
    for node in community:
        node_to_community[node] = i

# Loop over all papers
for i, paper in tqdm(df_abstract.iterrows()):
    # We need the authors and the tokens
    authors = []
    if type(paper['authors']) == str:
        authors = ast.literal_eval(paper['authors'])
    if type(paper['tokens']) == str:
        tokens = ast.literal_eval(paper['tokens'])

    # Loop over all authors of the paper
    for author in authors:
    # If the author is in the giant component, add the tokens to the community document
        if author in CSS_giant.nodes:
            community_documents[node_to_community[author]] += tokens
            titles_in_comm[node_to_community[author]].append(paper['title'])

# Save the community documents to a file
#np.save(DATA_PATH + 'community_documents.npy', community_documents)


35593it [00:06, 5915.83it/s]


In [19]:
#get top 5 largest communities and corresponding tokens
sorted_lists = sorted(zip(communities, community_documents), key=lambda x: len(x[0]))

sorted_communities, sorted_community_documents = zip(*sorted_lists)

In [23]:
len(sorted_communities[-5:])

5

In [89]:
token_counter_dict = {}
for i, comm_tokens in enumerate(reversed(sorted_community_documents[-15:])):
    token_counter_dict[i] = {}
    unique_tokens = list(set(comm_tokens))
    for unique_token in unique_tokens:
        token_counter_dict[i][unique_token] = 0


for i, comm_tokens in tqdm(enumerate(reversed(sorted_community_documents[-15:])), total = 15):
    unique_tokens = list(set(comm_tokens))
    for unique_token in unique_tokens:
        token_counter_dict[i][unique_token] += comm_tokens.count(unique_token)


100%|██████████| 15/15 [05:07<00:00, 20.53s/it]


In [86]:
def get_top_words_community(token_counter_dict, community, n_community):
    values = token_counter_dict[community].values()
    keys = token_counter_dict[community].keys()
    sorted_lists = sorted(list(zip(values, keys)), key=lambda x: x[0])
    values, keys = zip(*sorted_lists)

    return (values[-n_community:], keys[-n_community:])

In [87]:
for community in range(5):
    print(get_top_words_community(token_counter_dict, 0, 5))

((14326, 16423, 23205, 23567, 26353), ('in', 'to', 'and', 'of', 'the'))
((14326, 16423, 23205, 23567, 26353), ('in', 'to', 'and', 'of', 'the'))
((14326, 16423, 23205, 23567, 26353), ('in', 'to', 'and', 'of', 'the'))
((14326, 16423, 23205, 23567, 26353), ('in', 'to', 'and', 'of', 'the'))
((14326, 16423, 23205, 23567, 26353), ('in', 'to', 'and', 'of', 'the'))


#### Kan her ses at det er dårlig måde at adskille dem fordi der er for mange ord som er for generelle

In [62]:
term_in_doc_counter = {}
for i, paper in df_abstract.iterrows():
    if type(paper['tokens']) == str:
        tokens = set(ast.literal_eval(paper['tokens']))
        for token in tokens:
            try:
                term_in_doc_counter[token] += 1
            except:
                term_in_doc_counter[token] = 1

In [65]:
idf_dict = {}
n_documents = len(df_abstract) #though there are papers with no tokens...
for term in term_in_doc_counter.keys():
    idf_dict[term] = np.log(n_documents/term_in_doc_counter[term]) #Don't know which log to use, or the difference it makes


In [102]:
community_documents_counter = {}
for i, comm_titles in enumerate(titles_in_comm):
    community_documents_counter[i] = len(set(comm_titles))


In [101]:
sorted_lists = sorted(zip(communities, community_documents_counter), key=lambda x: len(x[0]))

sorted_communities, sorted_community_documents_counter = zip(*sorted_lists)

In [ ]:
term_in_doc_counter = {}
for i, paper in df_abstract.iterrows():
    if type(paper['tokens']) == str:
        tokens = set(ast.literal_eval(paper['tokens']))
        for token in tokens:
            try:
                term_in_doc_counter[token] += 1
            except:
                term_in_doc_counter[token] = 1